What we'll cover in this chapter
- What topics have been the most shared in the past year?
- What does the research on virality say?
- A look at headlines
- topic modelling and visualization
- cosine similarity

In [0]:
import requests
import pandas as pd
import numpy as np
import json
import time
from selenium import webdriver

pd.set_option('display.max_colwidth', 200)

In [0]:
browser = webdriver.PhantomJS()
browser.set_window_size(1080,800)
browser.get("http://www.ruzzit.com/en-US/Timeline?media=Articles&timeline=Year1&networks=All")
time.sleep(3)

pg_scroll_count = 50

while pg_scroll_count:
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(15)
    pg_scroll_count -= 1

titles = browser.find_elements_by_class_name("article_title")
link_class = browser.find_elements_by_class_name("link_read_more_article")
stats = browser.find_elements_by_class_name("ruzzit_statistics_area")

In [0]:
all_data = []
for title, link, stat in zip(titles, link_class, stats):
    all_data.append((title.text,\
                     link.get_attribute("href"),\
                     stat.find_element_by_class_name("col-md-12").text.split(' shares')[0],
                     stat.find_element_by_class_name("col-md-12").text.split('tweets\n')[1].split('likes\n0')[0],
                     stat.find_element_by_class_name("col-md-12").text.split('1\'s\n')[1].split(' pins')[0],
                     stat.find_element_by_class_name("col-md-12").text.split('pins\n')[1]))

In [0]:
all_data

[("Walker flap exposes GOP rift on Iowa's sacred status - CNN",
  'http://www.ruzzit.com/en-US/Redirect/Link?media=653892',
  '1.2M',
  '1,911 ',
  '8,693',
  '18/3/2015 18:41'),
 ('What Career Should You Actually Have? - BuzzFeed',
  'http://www.ruzzit.com/en-US/Redirect/Link?media=1928328',
  '5.1M',
  '1,559 ',
  '10.9k',
  '24/1/2016 21:00'),
 ('What State Do You Actually Belong In? - BuzzFeed',
  'http://www.ruzzit.com/en-US/Redirect/Link?media=1927663',
  '4.1M',
  '76 ',
  '5,465',
  '24/1/2016 15:15'),
 ('Which "Grease" Pink Lady Are You? - BuzzFeed',
  'http://www.ruzzit.com/en-US/Redirect/Link?media=1960941',
  '3M',
  '0 ',
  '2,760',
  '1/2/2016 03:46'),
 ('https://media.giphy.com/media/xTcnT6Kc5pbyZl8VzO/giphy.gif',
  'http://www.ruzzit.com/en-US/Redirect/Link?media=1931505',
  '2.4M',
  '17 ',
  '302',
  '15/1/2016 15:24'),
 ("Optimistic People All Have One Thing In Common: They're Always Late - Elite",
  'http://www.ruzzit.com/en-US/Redirect/Link?media=1082319',
  '1.6M'

In [0]:
df = pd.DataFrame(all_data, columns=['title', 'link', 'fb', 'lnkdn', 'pins', 'date'])
df

title  \
0                     Walker flap exposes GOP rift on Iowa's sacred status - CNN   
1                               What Career Should You Actually Have? - BuzzFeed   
2                               What State Do You Actually Belong In? - BuzzFeed   
3                                   Which "Grease" Pink Lady Are You? - BuzzFeed   
4                     https://media.giphy.com/media/xTcnT6Kc5pbyZl8VzO/giphy.gif   
5    Optimistic People All Have One Thing In Common: They're Always Late - Elite   
6    Fast Food Workers: You Don't Deserve $15 an Hour to Flip Burgers, and That'   
7         Zombie-themed 'Walking Dead' cruise setting sail in January - Fox News   
8                          https://media.giphy.com/media/9NAXpI52kjBjG/giphy.gif   
9     Man Builds 'Dog Train' To Take Rescued Pups Out On Little Adventures - The   
10                    https://media.giphy.com/media/3o8doO4sqbyDf1LjlC/giphy.gif   
11   https://49.media.tumblr.com/724a51968d0332bcaac2cc53b1a11516/tumblr_o0t70sd   
12   https://s-media-cache-ak0.pinimg.com/originals/ca/eb/17/caeb17ce45dd6b29634   
13                       15 hilarious parenting comics that are almost too real.   
14               33 Insanely Clever Things Your Small Apartment Needs - BuzzFeed   
15   Donald Trump has flip-flopped so much that Stephen Colbert hosted a Trump v   
16   That time 16-year-old Malala Yousafzai left Jon Stewart speechless with com   
17   21 Things You Only Know If You've Been Best Friends For 10 Years - BuzzFeed   
18   Husband Says He Can't Afford His Wife As A Stay-At-Home Mom And His ... - I   
19             18 Photos That Won't Make Sense To Sisterless Families - BuzzFeed   
20    Dad Picks Her Up From School And Finds Out She’s Been Bullied – Watch What   
21    Every September 11, I Remember This Surprising Story About Steve Buscemi -   
22                  No, It's Not Your Opinion. You're Just Wrong - Houston Press   
23   A Mom's Social Media Post Shatters Lives in AT&T's Gut-Wrenching Ad - Creat   
24      19 Vintage Easter Bunny Photos That Will Make Your Skin Crawl - BuzzFeed   
25                 CLICK HERE to support COMET: World's First Buoyant Smartphone   
26            London Fashion Week: Sibling Fall/Winter 2015 - The New York Times   
27   Dear white Facebook friends: I need you to respect what Black America is fe   
28    This Might Look Like A Nursing Home Or A Preschool, But It's Much Better -   
29     29 Dollar Store Finds That Will Keep Your Kids Busy All Summer - BuzzFeed   
..                                                                           ...   
480  A Harry Potter-Themed Bar Just Opened And You Have To Pay Your Respects - B   
481   Final Draft of Climate Change Accord to Be Released on Saturday - New York   
482    Tesla’s $3,000 Powerwall Will Let Households Run Entirely On Solar Energy   
483   I was held hostage by Isis. They fear our unity more than our airstrikes -   
484            Cold And Snow Predicted By Farmers' Almanac - The Huffington Post   
485               25 Pictures That Perfectly Sum Up Working In Retail - BuzzFeed   
486  A Former Planned Parenthood Employee Just Tweeted All the Horrifying Acts o   
487     The Families Funding the 2016 Presidential Election - The New York Times   
488           19 Small Awards Anyone With Anxiety Deserves To Receive - BuzzFeed   
489   There Will Be A Full Moon On Christmas For The First Time In Decades - The   
490  If You're Liberal and You Think Hillary Clinton Is Corrupt and Untrustworth   
491                 13 things mentally strong people don't do - Business Insider   
492  22 Things People Who Are Chronically Early Will Totally Understand - BuzzFe   
493           If Motivational Posters Were For People Who Hate People - BuzzFeed   
494   Mariah Carey Casually Sings Along To Her Greatest Hits With James Corden -   
495                     Angelina Jolie Pitt: Diary of a Surgery - New York Times   
496         Why My Husband Will Always Come

In [0]:
#browser.save_screenshot('/Users/alexcombs/Desktop/testimg.png')

True

In [0]:
df = df.assign(redirect = df['link'].map(lambda x: requests.get(x).url))

In [0]:
df

title  \
0                     Walker flap exposes GOP rift on Iowa's sacred status - CNN   
1                               What Career Should You Actually Have? - BuzzFeed   
2                               What State Do You Actually Belong In? - BuzzFeed   
3                                   Which "Grease" Pink Lady Are You? - BuzzFeed   
4                     https://media.giphy.com/media/xTcnT6Kc5pbyZl8VzO/giphy.gif   
5    Optimistic People All Have One Thing In Common: They're Always Late - Elite   
6    Fast Food Workers: You Don't Deserve $15 an Hour to Flip Burgers, and That'   
7         Zombie-themed 'Walking Dead' cruise setting sail in January - Fox News   
8                          https://media.giphy.com/media/9NAXpI52kjBjG/giphy.gif   
9     Man Builds 'Dog Train' To Take Rescued Pups Out On Little Adventures - The   
10                    https://media.giphy.com/media/3o8doO4sqbyDf1LjlC/giphy.gif   
11   https://49.media.tumblr.com/724a51968d0332bcaac2cc53b1a11516/tumblr_o0t70sd   
12   https://s-media-cache-ak0.pinimg.com/originals/ca/eb/17/caeb17ce45dd6b29634   
13                       15 hilarious parenting comics that are almost too real.   
14               33 Insanely Clever Things Your Small Apartment Needs - BuzzFeed   
15   Donald Trump has flip-flopped so much that Stephen Colbert hosted a Trump v   
16   That time 16-year-old Malala Yousafzai left Jon Stewart speechless with com   
17   21 Things You Only Know If You've Been Best Friends For 10 Years - BuzzFeed   
18   Husband Says He Can't Afford His Wife As A Stay-At-Home Mom And His ... - I   
19             18 Photos That Won't Make Sense To Sisterless Families - BuzzFeed   
20    Dad Picks Her Up From School And Finds Out She’s Been Bullied – Watch What   
21    Every September 11, I Remember This Surprising Story About Steve Buscemi -   
22                  No, It's Not Your Opinion. You're Just Wrong - Houston Press   
23   A Mom's Social Media Post Shatters Lives in AT&T's Gut-Wrenching Ad - Creat   
24      19 Vintage Easter Bunny Photos That Will Make Your Skin Crawl - BuzzFeed   
25                 CLICK HERE to support COMET: World's First Buoyant Smartphone   
26            London Fashion Week: Sibling Fall/Winter 2015 - The New York Times   
27   Dear white Facebook friends: I need you to respect what Black America is fe   
28    This Might Look Like A Nursing Home Or A Preschool, But It's Much Better -   
29     29 Dollar Store Finds That Will Keep Your Kids Busy All Summer - BuzzFeed   
..                                                                           ...   
480  A Harry Potter-Themed Bar Just Opened And You Have To Pay Your Respects - B   
481   Final Draft of Climate Change Accord to Be Released on Saturday - New York   
482    Tesla’s $3,000 Powerwall Will Let Households Run Entirely On Solar Energy   
483   I was held hostage by Isis. They fear our unity more than our airstrikes -   
484            Cold And Snow Predicted By Farmers' Almanac - The Huffington Post   
485               25 Pictures That Perfectly Sum Up Working In Retail - BuzzFeed   
486  A Former Planned Parenthood Employee Just Tweeted All the Horrifying Acts o   
487     The Families Funding the 2016 Presidential Election - The New York Times   
488           19 Small Awards Anyone With Anxiety Deserves To Receive - BuzzFeed   
489   There Will Be A Full Moon On Christmas For The First Time In Decades - The   
490  If You're Liberal and You Think Hillary Clinton Is Corrupt and Untrustworth   
491                 13 things mentally strong people don't do - Business Insider   
492  22 Things People Who Are Chronically Early Will Totally Understand - BuzzFe   
493           If Motivational Posters Were For People Who Hate People - BuzzFeed   
494   Mariah Carey Casually Sings Along To Her Greatest Hits With James Corden -   
495                     Angelina Jolie Pitt: Diary of a Surgery - New York Times   
496         Why My Husband Will Always Come

In [0]:
def check_home(x):
    if '.com' in x:
        if len(x.split('.com')[1]) < 2:
            return 1
        else:
            return 0
    else:
        return 0

In [0]:
def check_img(x):
    if '.gif' in x or '.jpg' in x:
        return 1
    else:
        return 0

In [0]:
df = df.assign(pg_missing = df['pg_missing'].map(check_home))

In [0]:
df = df.assign(img_link = df['redirect'].map(check_img))

In [0]:
df

title  \
0                     Walker flap exposes GOP rift on Iowa's sacred status - CNN   
1                               What Career Should You Actually Have? - BuzzFeed   
2                               What State Do You Actually Belong In? - BuzzFeed   
3                                   Which "Grease" Pink Lady Are You? - BuzzFeed   
4                     https://media.giphy.com/media/xTcnT6Kc5pbyZl8VzO/giphy.gif   
5    Optimistic People All Have One Thing In Common: They're Always Late - Elite   
6    Fast Food Workers: You Don't Deserve $15 an Hour to Flip Burgers, and That'   
7         Zombie-themed 'Walking Dead' cruise setting sail in January - Fox News   
8                          https://media.giphy.com/media/9NAXpI52kjBjG/giphy.gif   
9     Man Builds 'Dog Train' To Take Rescued Pups Out On Little Adventures - The   
10                    https://media.giphy.com/media/3o8doO4sqbyDf1LjlC/giphy.gif   
11   https://49.media.tumblr.com/724a51968d0332bcaac2cc53b1a11516/tumblr_o0t70sd   
12   https://s-media-cache-ak0.pinimg.com/originals/ca/eb/17/caeb17ce45dd6b29634   
13                       15 hilarious parenting comics that are almost too real.   
14               33 Insanely Clever Things Your Small Apartment Needs - BuzzFeed   
15   Donald Trump has flip-flopped so much that Stephen Colbert hosted a Trump v   
16   That time 16-year-old Malala Yousafzai left Jon Stewart speechless with com   
17   21 Things You Only Know If You've Been Best Friends For 10 Years - BuzzFeed   
18   Husband Says He Can't Afford His Wife As A Stay-At-Home Mom And His ... - I   
19             18 Photos That Won't Make Sense To Sisterless Families - BuzzFeed   
20    Dad Picks Her Up From School And Finds Out She’s Been Bullied – Watch What   
21    Every September 11, I Remember This Surprising Story About Steve Buscemi -   
22                  No, It's Not Your Opinion. You're Just Wrong - Houston Press   
23   A Mom's Social Media Post Shatters Lives in AT&T's Gut-Wrenching Ad - Creat   
24      19 Vintage Easter Bunny Photos That Will Make Your Skin Crawl - BuzzFeed   
25                 CLICK HERE to support COMET: World's First Buoyant Smartphone   
26            London Fashion Week: Sibling Fall/Winter 2015 - The New York Times   
27   Dear white Facebook friends: I need you to respect what Black America is fe   
28    This Might Look Like A Nursing Home Or A Preschool, But It's Much Better -   
29     29 Dollar Store Finds That Will Keep Your Kids Busy All Summer - BuzzFeed   
..                                                                           ...   
480  A Harry Potter-Themed Bar Just Opened And You Have To Pay Your Respects - B   
481   Final Draft of Climate Change Accord to Be Released on Saturday - New York   
482    Tesla’s $3,000 Powerwall Will Let Households Run Entirely On Solar Energy   
483   I was held hostage by Isis. They fear our unity more than our airstrikes -   
484            Cold And Snow Predicted By Farmers' Almanac - The Huffington Post   
485               25 Pictures That Perfectly Sum Up Working In Retail - BuzzFeed   
486  A Former Planned Parenthood Employee Just Tweeted All the Horrifying Acts o   
487     The Families Funding the 2016 Presidential Election - The New York Times   
488           19 Small Awards Anyone With Anxiety Deserves To Receive - BuzzFeed   
489   There Will Be A Full Moon On Christmas For The First Time In Decades - The   
490  If You're Liberal and You Think Hillary Clinton Is Corrupt and Untrustworth   
491                 13 things mentally strong people don't do - Business Insider   
492  22 Things People Who Are Chronically Early Will Totally Understand - BuzzFe   
493           If Motivational Posters Were For People Who Hate People - BuzzFeed   
494   Mariah Carey Casually Sings Along To Her Greatest Hits With James Corden -   
495                     Angelina Jolie Pitt: Diary of a Surgery - New York Times   
496         Why My Husband Will Always Come

In [0]:
df[df['pg_missing']==1]

title  \
0                     Walker flap exposes GOP rift on Iowa's sacred status - CNN   
26            London Fashion Week: Sibling Fall/Winter 2015 - The New York Times   
86              Israel and Palestine: Forgiving for Peace? - The Huffington Post   
118       Fighting Income Inequality Is at Hillary's Roots - The Huffington Post   
125               HACKED BY ISLAMIC STATE HACKING DIVISION - The Huffington Post   
150       3 Things That Can Hurt Your Business's Valuation - The Huffington Post   
153                    VIP Nightlife in DC Just Got Easier - The Huffington Post   
154                           3 Steps to Summer Ready Skin - The Huffington Post   
157                      Why Do We Grab Life By The Balls? - The Huffington Post   
162  5 Things Only People Sexually Attracted To Animals Know (NSFW) - The Huffin   
163              Win a Recording Session With Rock Legends - The Huffington Post   
164                  Shop The Look: Fun and Fab Home Décor - The Huffington Post   
165                                     Ready, Set, Smoke! - The Huffington Post   
172                      6 Reasons Why Airbnb is the Best. - The Huffington Post   
174                                The Frass Isn't Greener - The Huffington Post   
227          Pixels' is somehow even worse than I thought it could be - Engadget   
283                  5 Terrible Gifts for People Who Don't Value Money - Cracked   

                                                        link     fb   lnkdn  \
0     http://www.ruzzit.com/en-US/Redirect/Link?media=653892   1.2M  1,911    
26    http://www.ruzzit.com/en-US/Redirect/Link?media=558599   807k  8,290    
86   http://www.ruzzit.com/en-US/Redirect/Link?media=1390577   375k  3,660    
118  http://www.ruzzit.com/en-US/Redirect/Link?media=1949234   374k      3    
125   http://www.ruzzit.com/en-US/Redirect/Link?media=745234   368k      0    
150  http://www.ruzzit.com/en-US/Redirect/Link?media=1272113   330k      0    
153  http://www.ruzzit.com/en-US/Redirect/Link?media=1211957   328k      0    
154  http://www.ruzzit.com/en-US/Redirect/Link?media=1108704   322k  3,245    
157  http://www.ruzzit.com/en-US/Redirect/Link?media=1091700   322k  3,245    
162  http://www.ruzzit.com/en-US/Redirect/Link?media=1094859   322k      0    
163  http://www.ruzzit.com/en-US/Redirect/Link?media=1067969   321k      0    
164  http://www.ruzzit.com/en-US/Redirect/Link?media=1071318   321k      0    
165  http://www.ruzzit.com/en-US/Redirect/Link?media=1050711   321k      0    
172   http://www.ruzzit.com/en-US/Redirect/Link?media=913935   317k     65    
174   http://www.ruzzit.com/en-US/Redirect/Link?media=956714   317k      0    
227  http://www.ruzzit.com/en-US/Redirect/Link?media=1163823  28.6k  3,375    
283   http://www.ruzzit.com/en-US/Redirect/Link?media=762827   240k      1    

      pins             date                        redirect  pg_missing  \
0    8,693  18/3/2015 18:41             http://www.cnn.com/           1   
26   29.7k  21/2/2015 17:02         http://www.nytimes.com/           1   
86     46k  16/9/2015 23:47  http://www.huffingtonpost.com/           1   
118      0  29/1/2016 03:34  http://www.huffingtonpost.com/           1   
125      0  13/4/2015 19:27  http://www.huffingtonpost.com/           1   
150      0  19/8/2015 18:47  http://www.huffingtonpost.com/           1   
153      0   5/8/2015 02:55  http://www.huffingtonpost.com/           1   
154      0  10/7/2015 18:54  http://www.huffingtonpost.com/           1   
157      0   7/7/2015 02:30   http://www.huffingtonpost.com           1   
162      0   7/7/2015 20:20   http://www.huffingtonpost.com           1   
163      0   1/7/2015 02:55  http://www.huffingtonpost.com/           1   
164      0   1/7/2015 20:53  http://www.huffingtonpost.com/           1   
165      0  26/6/2015 18:38  http://www.huffingtonpost.com/           1   
172      0  23/5/2015 06:05  http://www.huffingtonpost.com/           1   
174      0   3

In [0]:
len(df[df['pg_missing']==1])

17

In [0]:
len(df[df['img_link']==1])

9

In [0]:
df[df['pg_missing']==1]

title  \
0                     Walker flap exposes GOP rift on Iowa's sacred status - CNN   
26            London Fashion Week: Sibling Fall/Winter 2015 - The New York Times   
86              Israel and Palestine: Forgiving for Peace? - The Huffington Post   
118       Fighting Income Inequality Is at Hillary's Roots - The Huffington Post   
125               HACKED BY ISLAMIC STATE HACKING DIVISION - The Huffington Post   
150       3 Things That Can Hurt Your Business's Valuation - The Huffington Post   
153                    VIP Nightlife in DC Just Got Easier - The Huffington Post   
154                           3 Steps to Summer Ready Skin - The Huffington Post   
157                      Why Do We Grab Life By The Balls? - The Huffington Post   
162  5 Things Only People Sexually Attracted To Animals Know (NSFW) - The Huffin   
163              Win a Recording Session With Rock Legends - The Huffington Post   
164                  Shop The Look: Fun and Fab Home Décor - The Huffington Post   
165                                     Ready, Set, Smoke! - The Huffington Post   
172                      6 Reasons Why Airbnb is the Best. - The Huffington Post   
174                                The Frass Isn't Greener - The Huffington Post   
227          Pixels' is somehow even worse than I thought it could be - Engadget   
283                  5 Terrible Gifts for People Who Don't Value Money - Cracked   

                                                        link     fb   lnkdn  \
0     http://www.ruzzit.com/en-US/Redirect/Link?media=653892   1.2M  1,911    
26    http://www.ruzzit.com/en-US/Redirect/Link?media=558599   807k  8,290    
86   http://www.ruzzit.com/en-US/Redirect/Link?media=1390577   375k  3,660    
118  http://www.ruzzit.com/en-US/Redirect/Link?media=1949234   374k      3    
125   http://www.ruzzit.com/en-US/Redirect/Link?media=745234   368k      0    
150  http://www.ruzzit.com/en-US/Redirect/Link?media=1272113   330k      0    
153  http://www.ruzzit.com/en-US/Redirect/Link?media=1211957   328k      0    
154  http://www.ruzzit.com/en-US/Redirect/Link?media=1108704   322k  3,245    
157  http://www.ruzzit.com/en-US/Redirect/Link?media=1091700   322k  3,245    
162  http://www.ruzzit.com/en-US/Redirect/Link?media=1094859   322k      0    
163  http://www.ruzzit.com/en-US/Redirect/Link?media=1067969   321k      0    
164  http://www.ruzzit.com/en-US/Redirect/Link?media=1071318   321k      0    
165  http://www.ruzzit.com/en-US/Redirect/Link?media=1050711   321k      0    
172   http://www.ruzzit.com/en-US/Redirect/Link?media=913935   317k     65    
174   http://www.ruzzit.com/en-US/Redirect/Link?media=956714   317k      0    
227  http://www.ruzzit.com/en-US/Redirect/Link?media=1163823  28.6k  3,375    
283   http://www.ruzzit.com/en-US/Redirect/Link?media=762827   240k      1    

      pins             date                        redirect  pg_missing  \
0    8,693  18/3/2015 18:41             http://www.cnn.com/           1   
26   29.7k  21/2/2015 17:02         http://www.nytimes.com/           1   
86     46k  16/9/2015 23:47  http://www.huffingtonpost.com/           1   
118      0  29/1/2016 03:34  http://www.huffingtonpost.com/           1   
125      0  13/4/2015 19:27  http://www.huffingtonpost.com/           1   
150      0  19/8/2015 18:47  http://www.huffingtonpost.com/           1   
153      0   5/8/2015 02:55  http://www.huffingtonpost.com/           1   
154      0  10/7/2015 18:54  http://www.huffingtonpost.com/           1   
157      0   7/7/2015 02:30   http://www.huffingtonpost.com           1   
162      0   7/7/2015 20:20   http://www.huffingtonpost.com           1   
163      0   1/7/2015 02:55  http://www.huffingtonpost.com/           1   
164      0   1/7/2015 20:53  http://www.huffingtonpost.com/           1   
165      0  26/6/2015 18:38  http://www.huffingtonpost.com/           1   
172      0  23/5/2015 06:05  http://www.huffingtonpost.com/           1   
174      0   3

In [0]:
dfc = df[(df['img_link']!=1)&(df['pg_missing']!=1)]

In [0]:
dfc

title  \
1                               What Career Should You Actually Have? - BuzzFeed   
2                               What State Do You Actually Belong In? - BuzzFeed   
3                                   Which "Grease" Pink Lady Are You? - BuzzFeed   
5    Optimistic People All Have One Thing In Common: They're Always Late - Elite   
6    Fast Food Workers: You Don't Deserve $15 an Hour to Flip Burgers, and That'   
7         Zombie-themed 'Walking Dead' cruise setting sail in January - Fox News   
9     Man Builds 'Dog Train' To Take Rescued Pups Out On Little Adventures - The   
13                       15 hilarious parenting comics that are almost too real.   
14               33 Insanely Clever Things Your Small Apartment Needs - BuzzFeed   
15   Donald Trump has flip-flopped so much that Stephen Colbert hosted a Trump v   
16   That time 16-year-old Malala Yousafzai left Jon Stewart speechless with com   
17   21 Things You Only Know If You've Been Best Friends For 10 Years - BuzzFeed   
18   Husband Says He Can't Afford His Wife As A Stay-At-Home Mom And His ... - I   
19             18 Photos That Won't Make Sense To Sisterless Families - BuzzFeed   
20    Dad Picks Her Up From School And Finds Out She’s Been Bullied – Watch What   
21    Every September 11, I Remember This Surprising Story About Steve Buscemi -   
22                  No, It's Not Your Opinion. You're Just Wrong - Houston Press   
23   A Mom's Social Media Post Shatters Lives in AT&T's Gut-Wrenching Ad - Creat   
24      19 Vintage Easter Bunny Photos That Will Make Your Skin Crawl - BuzzFeed   
25                 CLICK HERE to support COMET: World's First Buoyant Smartphone   
27   Dear white Facebook friends: I need you to respect what Black America is fe   
28    This Might Look Like A Nursing Home Or A Preschool, But It's Much Better -   
29     29 Dollar Store Finds That Will Keep Your Kids Busy All Summer - BuzzFeed   
30                5 reasons we can't handle marriage anymore - Asbury Park Press   
31       Why Modern Relationships Are Falling Apart So Easily Today - MensXP.com   
32   People Who Talk To Themselves Aren't Crazy, They're Actually Geniuses - Eli   
33   Watch: News Anchor Gets Fed Up With Obama, Says What Everyone's Thinking In   
34        27 Useful Dollar Store Finds Every Parent Should Know About - BuzzFeed   
35    A woman is knitting tiny wool jumpers to keep her chickens warm - Mashable   
36         24 Pictures That Will Make You Feel Better About The World - BuzzFeed   
..                                                                           ...   
480  A Harry Potter-Themed Bar Just Opened And You Have To Pay Your Respects - B   
481   Final Draft of Climate Change Accord to Be Released on Saturday - New York   
482    Tesla’s $3,000 Powerwall Will Let Households Run Entirely On Solar Energy   
483   I was held hostage by Isis. They fear our unity more than our airstrikes -   
484            Cold And Snow Predicted By Farmers' Almanac - The Huffington Post   
485               25 Pictures That Perfectly Sum Up Working In Retail - BuzzFeed   
486  A Former Planned Parenthood Employee Just Tweeted All the Horrifying Acts o   
487     The Families Funding the 2016 Presidential Election - The New York Times   
488           19 Small Awards Anyone With Anxiety Deserves To Receive - BuzzFeed   
489   There Will Be A Full Moon On Christmas For The First Time In Decades - The   
490  If You're Liberal and You Think Hillary Clinton Is Corrupt and Untrustworth   
491                 13 things mentally strong people don't do - Business Insider   
492  22 Things People Who Are Chronically Early Will Totally Understand - BuzzFe   
493           If Motivational Posters Were For People Who Hate People - BuzzFeed   
494   Mariah Carey Casually Sings Along To Her Greatest Hits With James Corden -   
495                     Angelina Jolie Pitt: Diary of a Surgery - New York Times   
496         Why My Husband Will Always Come

In [0]:
def get_data(x):
    try:
        data = requests.get('https://api.embedly.com/1/extract?key=SECRET_KEY&url=' + x)
        json_data = json.loads(data.text)
        return json_data
    except:
        print('Failed')
        return None

In [0]:
dfc = dfc.assign(json_data = dfc['redirect'].map(get_data))

In [0]:
dfc_bak = dfc

In [0]:
dfc

title  \
1                               What Career Should You Actually Have? - BuzzFeed   
2                               What State Do You Actually Belong In? - BuzzFeed   
3                                   Which "Grease" Pink Lady Are You? - BuzzFeed   
5    Optimistic People All Have One Thing In Common: They're Always Late - Elite   
6    Fast Food Workers: You Don't Deserve $15 an Hour to Flip Burgers, and That'   
7         Zombie-themed 'Walking Dead' cruise setting sail in January - Fox News   
9     Man Builds 'Dog Train' To Take Rescued Pups Out On Little Adventures - The   
13                       15 hilarious parenting comics that are almost too real.   
14               33 Insanely Clever Things Your Small Apartment Needs - BuzzFeed   
15   Donald Trump has flip-flopped so much that Stephen Colbert hosted a Trump v   
16   That time 16-year-old Malala Yousafzai left Jon Stewart speechless with com   
17   21 Things You Only Know If You've Been Best Friends For 10 Years - BuzzFeed   
18   Husband Says He Can't Afford His Wife As A Stay-At-Home Mom And His ... - I   
19             18 Photos That Won't Make Sense To Sisterless Families - BuzzFeed   
20    Dad Picks Her Up From School And Finds Out She’s Been Bullied – Watch What   
21    Every September 11, I Remember This Surprising Story About Steve Buscemi -   
22                  No, It's Not Your Opinion. You're Just Wrong - Houston Press   
23   A Mom's Social Media Post Shatters Lives in AT&T's Gut-Wrenching Ad - Creat   
24      19 Vintage Easter Bunny Photos That Will Make Your Skin Crawl - BuzzFeed   
25                 CLICK HERE to support COMET: World's First Buoyant Smartphone   
27   Dear white Facebook friends: I need you to respect what Black America is fe   
28    This Might Look Like A Nursing Home Or A Preschool, But It's Much Better -   
29     29 Dollar Store Finds That Will Keep Your Kids Busy All Summer - BuzzFeed   
30                5 reasons we can't handle marriage anymore - Asbury Park Press   
31       Why Modern Relationships Are Falling Apart So Easily Today - MensXP.com   
32   People Who Talk To Themselves Aren't Crazy, They're Actually Geniuses - Eli   
33   Watch: News Anchor Gets Fed Up With Obama, Says What Everyone's Thinking In   
34        27 Useful Dollar Store Finds Every Parent Should Know About - BuzzFeed   
35    A woman is knitting tiny wool jumpers to keep her chickens warm - Mashable   
36         24 Pictures That Will Make You Feel Better About The World - BuzzFeed   
..                                                                           ...   
480  A Harry Potter-Themed Bar Just Opened And You Have To Pay Your Respects - B   
481   Final Draft of Climate Change Accord to Be Released on Saturday - New York   
482    Tesla’s $3,000 Powerwall Will Let Households Run Entirely On Solar Energy   
483   I was held hostage by Isis. They fear our unity more than our airstrikes -   
484            Cold And Snow Predicted By Farmers' Almanac - The Huffington Post   
485               25 Pictures That Perfectly Sum Up Working In Retail - BuzzFeed   
486  A Former Planned Parenthood Employee Just Tweeted All the Horrifying Acts o   
487     The Families Funding the 2016 Presidential Election - The New York Times   
488           19 Small Awards Anyone With Anxiety Deserves To Receive - BuzzFeed   
489   There Will Be A Full Moon On Christmas For The First Time In Decades - The   
490  If You're Liberal and You Think Hillary Clinton Is Corrupt and Untrustworth   
491                 13 things mentally strong people don't do - Business Insider   
492  22 Things People Who Are Chronically Early Will Totally Understand - BuzzFe   
493           If Motivational Posters Were For People Who Hate People - BuzzFeed   
494   Mariah Carey Casually Sings Along To Her Greatest Hits With James Corden -   
495                     Angelina Jolie Pitt: Diary of a Surgery - New York Times   
496         Why My Husband Will Always Come

In [0]:
def get_title(x):
    try:
        return x.get('title')
    except:
        return None

In [0]:
dfc = dfc.assign(title = dfc['json_data'].map(get_title))

In [0]:
def get_site(x):
    try:
        return x.get('provider_name')
    except:
        return None

In [0]:
dfc = dfc.assign(site = dfc['json_data'].map(get_site))

In [0]:
def get_images(x):
    try:
        return len(x.get('images'))
    except:
        return None

In [0]:
dfc = dfc.assign(img_count = dfc['json_data'].map(get_images))

In [0]:
def get_entities(x):
    try:
        return [y.get('name') for y in x.get('entities')]
    except:
        return None

In [0]:
dfc = dfc.assign(entities = dfc['json_data'].map(get_entities))

In [0]:
def get_html(x):
    try:
        return x.get('content')
    except:
        return None

In [0]:
dfc = dfc.assign(html = dfc['json_data'].map(get_html))

In [0]:
dfc[::-1]

title  \
509                                                                                                      'Stay-at-Home Mom' Is Not a Real Job   
508                                                                                                    The Most Mysterious Star in Our Galaxy   
507                                                                                        Why Our Children Don't Think There Are Moral Facts   
506                                                                The Psychology Of Nail-Biting, Skin-Picking And Other Compulsive Behaviors   
505                                                  Yogi Berra, Yankee Who Built His Stardom 90 Percent on Skill and Half on Wit, Dies at 90   
504                                                                            10 Things to keep in Mind When Loving a Highly Creative Person   
503                                                    Archaeologists To Ben Carson: Ancient Egyptians Wrote Down Why The Pyramids Were Built   
502                                                                    Groom Makes Wedding Vows To 3-Year-Old Stepdaughter In Emotional Video   
501                                                                             Reporter regrets asking man what he'd do with Powerball money   
500                                                                                                                        A Toxic Work World   
499                                                                                14 Reasons Why The Irish Goodbye Is The Best Exit Strategy   
498                                                                       California Set To Give Solar Panels To Low-Income Families For Free   
497                                                                                                Supreme Court Ruling and Christian Outrage   
496                                                                                            Why My Husband Will Always Come Before My Kids   
495                                                                                                   Angelina Jolie Pitt: Diary of a Surgery   
494                                                                               Watch Mariah Carey Casually Sing Along To Her Greatest Hits   
493                                                                                   If Motivational Posters Were For People Who Hate People   
492                                                                        22 Things People Who Are Chronically Early Will Totally Understand   
491                                                                                                 13 things mentally strong people don't do   
490  If You're Liberal and You Think Hillary Clinton Is Corrupt and Untrustworthy, You're Rewarding 25 Years of GOP Smears - The Daily Banter   
489                                                                      There Will Be A Full Moon On Christmas For The First Time In Decades   
488                                                                                   19 Small Awards Anyone With Anxiety Deserves To Receive   
487                                                                                       The Families Funding the 2016 Presidential Election   
486                                       A Former Planned Parenthood Employee Just Tweeted All the Horrifying Acts of Terrorism She Survived   
485                                                                              25 Pictures That Will Give Retail Workers Intense Flashbacks   
484                                                                                           Cold And Snow Predicted By Old Farmers' Almanac   
483                                                  I was held hostage by Isis. They fear our unity more than our airstrikes | Nicolas Hénin   
482                                                                         

In [0]:
from bs4 import BeautifulSoup

In [0]:
def text_from_html(x):
    try:
        soup = BeautifulSoup(x, 'lxml')
        return soup.get_text()
    except:
        return None

In [0]:
dfc = dfc.assign(text = dfc['html'].map(text_from_html))

In [0]:
dfc[::-1]

title  \
509                                                                                                      'Stay-at-Home Mom' Is Not a Real Job   
508                                                                                                    The Most Mysterious Star in Our Galaxy   
507                                                                                        Why Our Children Don't Think There Are Moral Facts   
506                                                                The Psychology Of Nail-Biting, Skin-Picking And Other Compulsive Behaviors   
505                                                  Yogi Berra, Yankee Who Built His Stardom 90 Percent on Skill and Half on Wit, Dies at 90   
504                                                                            10 Things to keep in Mind When Loving a Highly Creative Person   
503                                                    Archaeologists To Ben Carson: Ancient Egyptians Wrote Down Why The Pyramids Were Built   
502                                                                    Groom Makes Wedding Vows To 3-Year-Old Stepdaughter In Emotional Video   
501                                                                             Reporter regrets asking man what he'd do with Powerball money   
500                                                                                                                        A Toxic Work World   
499                                                                                14 Reasons Why The Irish Goodbye Is The Best Exit Strategy   
498                                                                       California Set To Give Solar Panels To Low-Income Families For Free   
497                                                                                                Supreme Court Ruling and Christian Outrage   
496                                                                                            Why My Husband Will Always Come Before My Kids   
495                                                                                                   Angelina Jolie Pitt: Diary of a Surgery   
494                                                                               Watch Mariah Carey Casually Sing Along To Her Greatest Hits   
493                                                                                   If Motivational Posters Were For People Who Hate People   
492                                                                        22 Things People Who Are Chronically Early Will Totally Understand   
491                                                                                                 13 things mentally strong people don't do   
490  If You're Liberal and You Think Hillary Clinton Is Corrupt and Untrustworthy, You're Rewarding 25 Years of GOP Smears - The Daily Banter   
489                                                                      There Will Be A Full Moon On Christmas For The First Time In Decades   
488                                                                                   19 Small Awards Anyone With Anxiety Deserves To Receive   
487                                                                                       The Families Funding the 2016 Presidential Election   
486                                       A Former Planned Parenthood Employee Just Tweeted All the Horrifying Acts of Terrorism She Survived   
485                                                                              25 Pictures That Will Give Retail Workers Intense Flashbacks   
484                                                                                           Cold And Snow Predicted By Old Farmers' Almanac   
483                                                  I was held hostage by Isis. They fear our unity more than our airstrikes | Nicolas Hénin   
482                                                                         

## Clean up data for counts and date

In [0]:
def clean_counts(x):
    if 'M' in str(x):
        d = x.split('M')[0]
        dm = float(d) * 1000000
        return dm
    elif 'k' in str(x):
        d = x.split('k')[0]
        dk = float(d.replace(',','')) * 1000
        return dk
    elif ',' in str(x):
        d = x.replace(',','')
        return int(d)
    else:
        return x

In [0]:
dfc = dfc.assign(fb = dfc['fb'].map(clean_counts))

In [0]:
dfc = dfc.assign(lnkdn = dfc['lnkdn'].map(clean_counts))

In [0]:
dfc = dfc.assign(pins = dfc['pins'].map(clean_counts))

In [0]:
dfc = dfc.assign(date = pd.to_datetime(dfc['date'], dayfirst=True))

In [0]:
dfc

title  \
1                                                                                                       What Career Should You Actually Have?   
2                                                                                                       What State Do You Actually Belong In?   
3                                                                                                           Which "Grease" Pink Lady Are You?   
5                                                                         Optimistic People All Have One Thing In Common: They're Always Late   
6                                                             Fast Food Workers: You Don't Deserve $15 an Hour to Flip Burgers, and That's OK   
7                                                                                 Zombie-themed 'Walking Dead' cruise setting sail in January   
9                                                                        Man Builds 'Dog Train' To Take Rescued Pups Out On Little Adventures   
13                                                                                    15 hilarious parenting comics that are almost too real.   
14                                                                                       33 Insanely Clever Things Your Small Apartment Needs   
15                                                 Donald Trump has flip-flopped so much that Stephen Colbert hosted a Trump vs. Trump debate   
16                                                     16-Year-Old Malala Yousafzai Leaves Jon Stewart Speechless With Comment About Pacifism   
17                                                                           21 Things You Know Only If You've Been Best Friends For 10 Years   
18                                           Husband Says He Can't Afford His Wife As A Stay-At-Home Mom And His Reasoning Adds Up. Big Time.   
19                                                                                     18 Photos That Won't Make Sense To Sisterless Families   
20                                                         Dad Picks Her Up From School And Finds Out She's Been Bullied - Watch What He Does   
21                                                                   Every September 11, I Remember This Surprising Story About Steve Buscemi   
22                                                                                     No, It's Not Your Opinion. You're Just Wrong [UPDATED]   
23                                               Best of 2015 #8 TV/Film: A Mom's Social Media Post Shatters Lives in AT&T's Gut-Wrenching Ad   
24                                                                              19 Vintage Easter Bunny Photos That Will Make Your Skin Crawl   
25                                                                             CLICK HERE to support COMET: World's First Floating Smartphone   
27                                                 Dear white Facebook friends: I need you to respect what Black America is feeling right now   
28                                                                   They Put A Pre-School In A Nursing Home, And It Changed Everyone's Lives   
29                                                                             29 Dollar-Store Finds That Will Keep Your Kids Busy All Summer   
30                                                                                                 5 reasons we can't handle marriage anymore   
31                                                                                 Why Modern Relationships Are Falling Apart So Easily Today   
32                                                                      People Who Talk To Themselves Aren't Crazy, They're Actually Geniuses   
33                                                      Watch: News Anchor Gets Fed Up With Obama, Says What Everyone's Thinking In EPIC Rant   
34                                                                          

In [0]:
def get_word_count(x):
    if not x is None:
        return len(x.split(' '))
    else:
        return None

In [0]:
dfc = dfc.assign(word_count = dfc['text'].map(get_word_count))

In [0]:
dfc[['text','word_count']][::-1]

text  \
99                                                                                                                                                                                                      None   
98   \nAre you smarter than a Common Core 4th grader?\nTom Hinchey March 6, 2015 at 12:14pm\nAdvertisement - story continues below\nKaren Lamoreaux addressed the Arkansas Board of Education in December...   
97   \n\n\n \n\n\n \n\n\n\n\n1. The theme song says the word Caillou 14,356 times. And it's sung by him, in his voice, which will drive you nuts after five minutes. The director basically told the voic...   
96   \n4. There was a smoking section in the dining hall. \n16. And you could call anyone on campus just by dialing their "extension." \n17. When you got your roommate assignment, you had NO IDEA what ...   
95                                                                                                                                                                                                      None   
94   \nWe now know why the United States Congress absolutely did not want Pope Francis to come to America. He makes them look like the complete as*holes they are. Obviously, Congress thought Pope Franc...   
93   \nGiven the historic low temperatures and snowfalls that pummeled the eastern U.S. this winter, it might be easy to overlook how devastating California's winter was as well.\nAs our "wet" season d...   
92   \n\nNicholas Winton, a Briton who said nothing for a half-century about his role in organizing the escape of 669 mostly Jewish children from Czechoslovakia on the eve of World War II, a righteous ...   
91                                                                                                                                                                                                      None   
90   \nHistorians may look to 2015 as the year when shit really started hitting the fan. Some snapshots: In just the past few months, record-setting heat waves in Pakistan and India each killed more th...   
9    \nEugene Bostick may have officially retired about 15 years ago, but in some ways that was when his most impactful work began.\nNot long after, he embarked on a new career path of sorts - as a tra...   
89   \nAccording to PBS, Bernie Sanders is "gaining against Clinton in early polls." Salon's Bill Curry believes "Hillary Clinton is going lose," primarily because millions of voters longing for a trul...   
88   \nMeet The Essential Oil That Could Stop Cancer In Its Tracks\nEven scientists now recognize the power agents that exist within some essential oils which stops cancer spreading, and which induces ...   
87   \nLast fall, I wrote about a young man named Kalief Browder, who spent three years on Rikers Island without being convicted of a crime. He had been arrested in the spring of 2010, at age sixteen, ...   
85   \nOne of the things the brilliant minds at MIT do - besides ponder the nature of the universe and build sci-fi gizmos, of course - is notarize aircraft airworthiness for the federal government. So...   
84   \nWhen we were young, outside wasn't just the best option, it was the  only option. It was the only option I considered anyway. Still years away from home computers and social media, we had marble...   
83                                                                                                                                                                                                      None   
82   \nThe city of Baltimore has been besieged by riots Monday night - and police are on the scene ready to serve, protect and subdue.\nThis has become an evergreen narrative in the aftermath of reacti...   
81   \nEditor's Note: Anthony D'Ambrosio, 29, has built a large following after the success of his relationship columns that regularly appear on these pages. Today, he discusses why marriages just don'...   
80                                      

## Get Main Image Colors

In [0]:
import matplotlib.colors as mpc

In [0]:
def get_hex(x):
    try:
        if x.get('images'):
            main_color = x.get('images')[0].get('colors')[0].get('color')
            return mpc.rgb2hex([(x/255) for x in main_color])
    except:
        return None

In [0]:
def get_rgb(x):
    try:
        if x.get('images'):
            main_color = x.get('images')[0].get('colors')[0].get('color')
            return main_color
    except:
        return None

In [0]:
dfc = dfc.assign(main_hex = dfc['json_data'].map(get_hex))
dfc = dfc.assign(main_rgb = dfc['json_data'].map(get_rgb))

In [0]:
dfc

date       fb  \
1   2016-01-24 21:00:00  5100000   
100 2015-02-23 15:39:00   400000   
101 2015-07-11 00:36:00   402000   
102 2015-06-26 22:08:00   401000   
103 2015-07-15 07:15:00   402000   
104 2015-09-30 22:56:00   398000   
105 2015-06-23 22:58:00   395000   
106 2015-07-01 18:30:00   392000   
107 2015-04-01 23:16:00   391000   
108 2015-08-21 19:40:00   383000   
109 2015-10-15 20:46:00   386000   
110 2015-02-24 23:00:00   383000   
111 2015-11-06 18:10:00   383000   
112 2015-09-25 17:13:00   382000   
113 2015-02-25 23:17:00   373000   
114 2015-03-16 23:20:00   378000   
115 2015-06-17 16:01:00   378000   
116 2015-10-08 21:15:00   369000   
117 2016-02-04 19:53:00   374000   
119 2015-02-20 01:46:00   371000   
120 2015-07-30 04:22:00   359000   
121 2015-07-29 21:38:00   369000   
122 2015-05-28 01:30:00   368000   
123 2015-09-20 19:05:00   352000   
124 2015-12-05 03:13:00   366000   
126 2015-06-09 16:31:00   362000   
127 2015-03-15 15:33:00   363000   
128 2015-03-09 23:31:00   364000   
129 2015-09-12 03:01:00   360000   
13  2015-10-18 21:58:00  1100000   
..                  ...      ...   
70  2015-07-05 00:05:00   471000   
71  2016-02-02 01:15:00   473000   
72  2015-08-18 17:23:00   466000   
73  2015-07-07 23:10:00   465000   
74  2015-09-30 15:29:00   460000   
75  2015-04-11 20:30:00   436000   
76  2015-10-20 13:03:00   455000   
77  2015-10-25 13:16:00   447000   
78  2015-09-28 23:55:00   448000   
79  2015-10-14 00:42:00   441000   
80  2016-01-27 14:50:00   448000   
81  2015-04-06 19:52:00   433000   
82  2015-04-28 02:44:00   427000   
83  2015-06-18 03:46:00   431000   
84  2015-04-18 20:41:00   428000   
85  2016-01-12 06:05:00   429000   
87  2015-06-08 02:09:00   402000   
88  2015-06-20 14:20:00   416000   
89  2015-06-25 17:18:00   417000   
9   2015-09-22 23:02:00  1200000   
90  2015-08-05 16:25:00   402000   
91  2015-04-06 23:56:00   410000   
92  2015-07-01 18:31:00   401000   
93  2015-03-13 02:27:00   397000   
94  2015-09-24 02:43:00   407000   
95  2015-02-26 03:18:00   410000   
96  2015-08-07 01:16:00   409000   
97  2015-11-18 00:36:00   407000   
98  2015-03-05 23:21:00   406000   
99  2015-08-02 21:00:00   403000   

                                                                                                                                                                                                        html  \
1                                                                                                                                                                                                       None   
100  <div>\n<p>\n</p><p><a href="http://www.buzzfeed.com/trending"> <img src="https://img.buzzfeed.com/buzzfeed-static/static/badge_images/viral.png?v=1455385914"></a></p>\n<div>\n<div>\n<div>\n<div>1....   
101  <div>\n<p>\n</p><div>\n<div>\n<div>\n<div>1. Everyone loves ice cream. </div>\n<div>2. And of course churros are just as incredible. </div>\n<div>\n<h2>3. Individually, they make for a pretty tast...   
102  <div>\n<h3>In 2013, Zerlina Maxwell ignited a firestorm of controversy when she strongly recommended we stop telling women how to not get raped. </h3>\n<p>Here are her words, from the <a href="htt...   
103                                                                                                                                                                                                     None   
104                                                                                                                                                                                                     None   
105  <div>\n<p>Despite that, Congress still hasn't found the will to deal with gun violence in the U.S. Suffice it to say a lot of people are angry. Including the pile of celebrities you'll see below.<...   
106  <div>\n<div>\n<p><a href="http://www.buzzfeed.com/trending"> <img src="https://img.buzzfeed.com/buzzfeed-static/stati

In [0]:
dfc['img_count'].value_counts().to_frame('count')

count
5    342
4     37
2     36
1     36
3     30
0      1

In [0]:
fig, ax = plt.subplots(figsize=(8,6))
y = dfc['img_count'].value_counts().sort_index()
x = y.sort_index().index
plt.bar(x, y, color='k', align='center')
plt.title('Image Count Frequency', fontsize=16, y=1.01)
ax.set_xlim(-.5,5.5)
ax.set_ylabel('Count')
ax.set_xlabel('Number of Images')

In [0]:
#dfc.to_json('/Users/alexcombs/Desktop/viral_data.json')
dfc = pd.read_json('/Users/alexcombs/Desktop/viral_data.json')

In [0]:
mci = dfc['main_hex'].value_counts().to_frame('count')
mci

count
#f8fbfa      3
#c4c4c4      3
#211537      2
#3dd876      2
#39546c      2
#070603      2
#f1f3f1      2
#f6fafb      2
#c6b7b5      2
#f4f8f9      2
#312c27      2
#eaeeef      2
#eaeff2      1
#141914      1
#dad8d4      1
#594e4c      1
#948b5b      1
#f0f1e9      1
#b2c9c8      1
#030407      1
#8a9398      1
#08121b      1
#998e7e      1
#b0aca9      1
#ab997b      1
#bd8276      1
#d8ebf8      1
#746046      1
#dde2e1      1
#181719      1
...        ...
#e9eff4      1
#dee4d8      1
#0d1015      1
#dadddc      1
#c1d1d7      1
#c8cfd5      1
#e1e4df      1
#b4b8bb      1
#bfbdca      1
#1e3f53      1
#edf4f4      1
#050608      1
#9aa0a1      1
#191917      1
#61615e      1
#180f09      1
#ebf0f3      1
#514c3d      1
#091015      1
#595051      1
#f6f9f8      1
#f0f6f7      1
#1e1f1f      1
#242422      1
#b7b2af      1
#3772a1      1
#0d1720      1
#4e2630      1
#5e5b4f      1
#657d97      1

[450 rows x 1 columns]

In [0]:
mci['color'] = ' '

In [0]:
def color_cells(x):
    return 'background-color: ' + x.index

In [0]:
mci.style.apply(color_cells, subset=['color'], axis=0)

In [0]:
def get_csplit(x):
    try:
        return x[0], x[1], x[2]
    except:
        return None, None, None

In [0]:
dfc['reds'], dfc['greens'], dfc['blues'] = zip(*dfc['main_rgb'].map(get_csplit))

In [0]:
dfc

title  \
1                                                                                                       What Career Should You Actually Have?   
2                                                                                                       What State Do You Actually Belong In?   
3                                                                                                           Which "Grease" Pink Lady Are You?   
5                                                                         Optimistic People All Have One Thing In Common: They're Always Late   
6                                                             Fast Food Workers: You Don't Deserve $15 an Hour to Flip Burgers, and That's OK   
7                                                                                 Zombie-themed 'Walking Dead' cruise setting sail in January   
9                                                                        Man Builds 'Dog Train' To Take Rescued Pups Out On Little Adventures   
13                                                                                    15 hilarious parenting comics that are almost too real.   
14                                                                                       33 Insanely Clever Things Your Small Apartment Needs   
15                                                 Donald Trump has flip-flopped so much that Stephen Colbert hosted a Trump vs. Trump debate   
16                                                     16-Year-Old Malala Yousafzai Leaves Jon Stewart Speechless With Comment About Pacifism   
17                                                                           21 Things You Know Only If You've Been Best Friends For 10 Years   
18                                           Husband Says He Can't Afford His Wife As A Stay-At-Home Mom And His Reasoning Adds Up. Big Time.   
19                                                                                     18 Photos That Won't Make Sense To Sisterless Families   
20                                                         Dad Picks Her Up From School And Finds Out She's Been Bullied - Watch What He Does   
21                                                                   Every September 11, I Remember This Surprising Story About Steve Buscemi   
22                                                                                     No, It's Not Your Opinion. You're Just Wrong [UPDATED]   
23                                               Best of 2015 #8 TV/Film: A Mom's Social Media Post Shatters Lives in AT&T's Gut-Wrenching Ad   
24                                                                              19 Vintage Easter Bunny Photos That Will Make Your Skin Crawl   
25                                                                             CLICK HERE to support COMET: World's First Floating Smartphone   
27                                                 Dear white Facebook friends: I need you to respect what Black America is feeling right now   
28                                                                   They Put A Pre-School In A Nursing Home, And It Changed Everyone's Lives   
29                                                                             29 Dollar-Store Finds That Will Keep Your Kids Busy All Summer   
30                                                                                                 5 reasons we can't handle marriage anymore   
31                                                                                 Why Modern Relationships Are Falling Apart So Easily Today   
32                                                                      People Who Talk To Themselves Aren't Crazy, They're Actually Geniuses   
33                                                      Watch: News Anchor Gets Fed Up With Obama, Says What Everyone's Thinking In EPIC Rant   
34                                                                          

In [0]:
from sklearn.cluster import KMeans

In [0]:
np.sqrt(256)

16.0

In [0]:
clf = KMeans(n_clusters=16)

In [0]:
clf.fit(dfc[['reds', 'greens', 'blues']].dropna())

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=16, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

In [0]:
clusters = pd.DataFrame(clf.cluster_centers_, columns=['r', 'g', 'b'])

In [0]:
clusters

r           g           b
0   191.235294  161.705882  135.941176
1    32.825397   31.507937   36.603175
2   213.357143  217.607143  215.017857
3   108.583333  105.000000   94.000000
4    82.583333  145.083333  152.666667
5    13.533333   14.733333   17.422222
6   238.509091  242.472727  242.309091
7     1.600000   82.000000  156.200000
8   132.714286   56.428571   30.857143
9    79.842105   69.026316   63.473684
10  176.157895  190.315789  196.263158
11  211.888889  189.555556  170.611111
12  149.818182  136.227273  111.136364
13  144.434783  154.478261  156.521739
14  217.250000   55.250000   50.750000
15   52.545455   72.636364  103.272727

In [0]:
def hexify(x):
    rgb = [round(x['r']), round(x['g']), round(x['b'])]
    hxc = mpc.rgb2hex([(x/255) for x in rgb])
    return hxc

In [0]:
clusters.index = clusters.apply(hexify, axis=1)

In [0]:
clusters

r           g           b
#bfa288  191.235294  161.705882  135.941176
#212025   32.825397   31.507937   36.603175
#d5dad7  213.357143  217.607143  215.017857
#6d695e  108.583333  105.000000   94.000000
#539199   82.583333  145.083333  152.666667
#0e0f11   13.533333   14.733333   17.422222
#eff2f2  238.509091  242.472727  242.309091
#02529c    1.600000   82.000000  156.200000
#85381f  132.714286   56.428571   30.857143
#50453f   79.842105   69.026316   63.473684
#b0bec4  176.157895  190.315789  196.263158
#d4beab  211.888889  189.555556  170.611111
#96886f  149.818182  136.227273  111.136364
#909a9d  144.434783  154.478261  156.521739
#d93733  217.250000   55.250000   50.750000
#354967   52.545455   72.636364  103.272727

In [0]:
clusters['color'] = ' '

In [0]:
clusters

r           g           b color
#bfa288  191.235294  161.705882  135.941176      
#212025   32.825397   31.507937   36.603175      
#d5dad7  213.357143  217.607143  215.017857      
#6d695e  108.583333  105.000000   94.000000      
#539199   82.583333  145.083333  152.666667      
#0e0f11   13.533333   14.733333   17.422222      
#eff2f2  238.509091  242.472727  242.309091      
#02529c    1.600000   82.000000  156.200000      
#85381f  132.714286   56.428571   30.857143      
#50453f   79.842105   69.026316   63.473684      
#b0bec4  176.157895  190.315789  196.263158      
#d4beab  211.888889  189.555556  170.611111      
#96886f  149.818182  136.227273  111.136364      
#909a9d  144.434783  154.478261  156.521739      
#d93733  217.250000   55.250000   50.750000      
#354967   52.545455   72.636364  103.272727

In [0]:
clusters.style.apply(color_cells, subset=['color'], axis=0)

In [0]:
dfc[dfc['title'].isnull()]

date      fb  html  img_count  img_link  \
482 2015-05-01 07:03:00  181000  None        NaN         0   
69  2015-06-04 18:00:00  481000  None        NaN         0   

                                                                                                                    json_data  \
482                                              {'error_message': 'HTTP 404: Not Found', 'error_code': 404, 'type': 'error'}   
69   {'error_message': 'Only allowed one "url" parameter. Use "urls" for multiple urls.', 'error_code': 400, 'type': 'error'}   

                                                       link  lnkdn main_hex  \
482  http://www.ruzzit.com/en-US/Redirect/Link?media=818225   2508     None   
69   http://www.ruzzit.com/en-US/Redirect/Link?media=961526      3     None   

    main_rgb  pg_missing  pins  \
482     None           0     0   
69      None           0     0   

                                                                  redirect  \
482  http://social.techcrunch.com/2015/04/30/tesla-powerwall-home-battery/   
69       http://www.peoplepets.com/people/pets/article/0,,20928208,00.html   

     site  text title  word_count  
482  None  None  None         NaN  
69   None  None  None         NaN

## Headline Analysis

In [0]:
from nltk.util import ngrams
from nltk.corpus import stopwords
import re

def get_word_stats(txt_series, n, rem_stops=False):
    txt_words = []
    txt_len = []
    for w in txt_series:
        if w is not None:
            if rem_stops == False:
                word_list = [x for x in ngrams(re.findall('[a-z0-9\']+', w.lower()), n)]
            else:
                word_list = [y for y in ngrams([x for x in re.findall('[a-z0-9\']+', w.lower())\
                                                if x not in stopwords.words('english')], n)]
            word_list_len = len(list(word_list))
            txt_words.extend(word_list)
            txt_len.append(word_list_len)
    return pd.Series(txt_words).value_counts().to_frame('count'), pd.DataFrame(txt_len, columns=['count'])

In [0]:
hw,hl = get_word_stats(dfc['title'], 3, 1)

In [0]:
hw

count
(pictures, real, people)             3
(ring, fire, network)                3
(things, you'll, understand)         3
(full, moon, christmas)              2
(marines, dies, 83)                  2
(gay, marriage, putting)             2
(talking, white, people)             2
(rescue, baby, trapped)              2
(putting, country, vanguard)         2
(first, black, general)              2
(car, fox, news)                     2
(votes, approve, gay)                2
(baby, trapped, inside)              2
(handle, marriage, anymore)          2
(marriage, putting, country)         2
(officers, say, mysterious)          2
(petersen, first, black)             2
(trapped, inside, car)               2
(called, rescue, baby)               2
(dollar, store, finds)               2
(mysterious, voice, called)          2
(e, petersen, first)                 2
(can't, handle, marriage)            2
(utah, officers, say)                2
(approve, gay, marriage)             2
(people, talking, white)             2
(ireland, votes, approve)            2
(say, mysterious, voice)             2
(black, people, talking)             2
(inside, car, fox)                   2
...                                ...
(adds, new, feature)                 1
(android, apps, google)              1
(terminally, ill, patients)          1
(11, reasons, crazy)                 1
(fucking, things, women)             1
(crazy, they're, actually)           1
(american, life, early)              1
(surprise, deaf, neighbor)           1
(called, 'mentally, ill')            1
(farming, one, worst)                1
(parent, wrote, elementary)          1
(playhouse, brings, back)            1
(25, life, changing)                 1
(trump, flip, flopped)               1
(production, provide, water)         1
(wants, josh, duggar)                1
(obama, couldn't, even)              1
(offers, eye, opening)               1
(leaders, let's, bury)               1
(later, drop, bombshell)             1
(gets, two, years)                   1
(weather, every, day)                1
(never, ever, let)                   1
(elementary, school, hilarious)      1
(said, goodbye, paul)                1
(awful, cost, getting)               1
(flip, burgers, that's)              1
(new, study, finds)                  1
(owned, dog, relate)                 1
(donald, trump, flip)                1

[2471 rows x 1 columns]

In [0]:
hl.describe()

count
count  482.000000
mean    10.948133
std      3.436294
min      1.000000
25%      9.000000
50%     11.000000
75%     13.000000
max     25.000000

In [0]:
tt = dfc[~dfc['title'].isnull()]

In [0]:
tt[tt['title'].str.contains('Dies')]

date      fb  \
158 2015-05-15 08:21:00  303000   
326 2015-02-27 00:01:00  217000   
327 2015-11-15 11:51:00  220000   
368 2015-02-27 18:10:00  135000   
374 2015-08-27 02:06:00  207000   
376 2015-08-27 02:06:00  207000   
424 2015-08-30 11:07:00  177000   
505 2015-09-23 08:58:00  172000   
92  2015-07-01 18:31:00  401000   

                                                                                                                                                                                                        html  \
158  <div>\n<p><figure><img src="http://static01.nyt.com/images/2015/05/09/arts/music/00king-bb-adv-obit-slide-PKN2/00king-bb-adv-obit-slide-PKN2-jumbo.jpg"></figure></p>\n<p>B. B. King, whose world-we...   
326  <div>\n<p>"They weren't fancy. They were just decent people that were always committed to each other, no matter the situation."</p>\n<p>That's how Donna Scharton remembers her beloved parents, Flo...   
327  <div>\n<figure><img src="http://graphics8.nytimes.com/images/2015/11/11/us/scorah-karl-motherlode/scorah-karl-motherlode-tmagArticle.jpg"></figure><p>As he did most mornings of the 117 days of his...   
368  <div>\n<p>Leonard Nimoy, the sonorous, gaunt-faced actor who won a worshipful global following as Mr. Spock, the resolutely logical human-alien first officer of the Starship Enterprise in the tele...   
374  <div>\n<p><figure><img src="http://static01.nyt.com/images/2015/08/27/us/27petersen-obit/27petersen-obit-blog427.jpg"></figure></p>\n<p>Frank E. Petersen Jr., who suffered bruising racial indignit...   
376  <div>\n<p><figure><img src="http://static01.nyt.com/images/2015/08/27/us/27petersen-obit/27petersen-obit-blog427.jpg"></figure></p>\n<p>Frank E. Petersen Jr., who suffered bruising racial indignit...   
424  <div>\n<p>Oliver Sacks, the neurologist and acclaimed author who explored some of the brain's strangest pathways in best-selling case histories like "The Man Who Mistook His Wife for a Hat," using...   
505  <div>\n<p><figure><a href="http://www.nytimes.com/slideshow/2015/09/23/obituaries/yogi-berra-baseball-catcher-and-character-dies.html"> <img src="http://static01.nyt.com/images/2013/01/03/obituari...   
92   <div>\n<p><figure><img src="http://static01.nyt.com/images/2015/07/02/world/02wintonobit3/02wintonobit3-master675.jpg"></figure></p>\n<p>Nicholas Winton, a Briton who said nothing for a half-centu...   

     img_count  img_link  \
158          4         0   
326          2         0   
327          5         0   
368          4         0   
374          2         0   
376          2         0   
424          5         0   
505          5         0   
92           5         0   

                                                                                                                                                                                                   json_data  \
158  {'offset': None, 'original_url': 'http://www.nytimes.com/2015/05/16/arts/music/b-b-king-blues-singer-dies-at-89.html?_r=0', 'cache_age': 78736, 'published': 1431648000000, 'related': [], 'language...   
326  {'offset': -18000000, 'original_url': 'http://abcnews.go.com/Lifestyle/real-life-notebook-couple-dies-hand-hand/story?id=29226870', 'cache_age': 79145, 'published': 1424903940000, 'related': [], '...   
327  {'offset': None, 'original_url': 'http://parenting.blogs.nytimes.com/2015/11/15/a-baby-dies-at-day-care-and-a-mother-asks-why-she-had-to-leave-him-so-soon/?_r=0', 'cache_age': 79149, 'published': ...   
368  {'offset': None, 'original_url': 'http://www.nytimes.com/2015/02/27/arts/television/leonard-nimoy-spock-of-star-trek-dies-at-83.html?_r=0', 'cache_age': 13367, 'published': 1424995200000, 'related...   
374  {'offset': None, 'original_url': 'http://www.nytimes.com/2015/08/27/us/frank-e-petersen-first-black-general-in-marines-dies-at-83.html?_r=0', 'cache_age': 79250, 'published': 1440547200000, 'relat...   
376  {'offset': None, 'original_url': 'http://www.nytimes.

In [0]:
dfc['site'].value_counts().to_frame()

site
BuzzFeed                                                             131
The Huffington Post                                                   56
Nytimes                                                               35
Upworthy                                                              24
IFLScience                                                            20
Washington Post                                                       15
Mashable                                                              13
Mic                                                                   11
Western Journalism                                                     8
Business Insider                                                       8
the Guardian                                                           6
CNN                                                                    6
The Atlantic                                                           6
BuzzFeed Community                                                     5
Fox News                                                               5
Rolling Stone                                                          4
Mail Online                                                            4
Cracked.com                                                            4
TODAY.com                                                              4
Breitbart                                                              3
PEOPLE.com                                                             3
Vox                                                                    3
Elite Daily                                                            3
The Blaze                                                              3
The Ring of Fire Network                                               3
Quartz                                                                 3
TIME                                                                   2
ScienceAlert                                                           2
National Geographic News                                               2
Forbes                                                                 2
...                                                                  ...
The Daily Banter                                                       1
OZY                                                                    1
DNAinfo New York                                                       1
PBS NewsHour                                                           1
theCHIVE                                                               1
The Daily Signal                                                       1
Windsor News - Breaking News &amp; Latest Headlines | Windsor Star     1
For The Win                                                            1
The New Yorker                                                         1
CityLab                                                                1
Dorkly                                                                 1
Inc.com                                                                1
Bloomberg.com                                                          1
Newschannel10                                                          1
Christiantoday                                                         1
I Heart Intelligence                                                   1
Boredom Therapy                                                        1
NextShark                                                              1
ScienceDaily                                                           1
Motherlode Blog                                                        1
Twitter                                                                1
Express.co.uk                                                          1
Vanity Fair                                                            1
Wake Up World                                        

## Examine the Body Content

In [0]:
hw,hl = get_word_stats(dfc['text'], 3, 1)

In [0]:
hw

count
(advertisement, story, continues)      32
(articles, buzzfeed, com)              27
(check, articles, buzzfeed)            27
(buzzfeed, com, tagged)                21
(new, york, times)                     19
(via, upward, spiral)                  17
(pic, twitter, com)                    17
(new, york, city)                      16
(every, single, day)                   16
(follow, us, twitter)                  15
(like, us, facebook)                   14
(g, m, o)                              13
(facebook, follow, us)                 13
(us, facebook, follow)                 13
(may, like, conversations)             12
(playstation, 5, xbox)                 12
(5, xbox, two)                         12
(april, 15, 2015)                      10
(25, year, old)                        10
(told, huffington, post)               10
(stay, home, mom)                      10
(12, year, old)                         9
(something, add, story)                 9
(1, 5, oz)                              9
(add, story, share)                     9
(iowa, new, hampshire)                  9
(story, share, comments)                9
(us, twitter, contact)                  8
(twitter, contact, huffpost)            8
(15, 2015, mattwalshblog)               8
...                                   ...
(go, ease, reap)                        1
(human, counterparts, days)             1
(blossom, pups, increased)              1
(picking, women, without)               1
(right, personally, case)               1
(instagram, misty, take)                1
(intellectual, property, 1962)          1
(night, close, contact)                 1
(oprah, airs, sundays)                  1
(fit, death, eaters)                    1
(sophisticated, deadly, strikes)        1
(sale, runs, june)                      1
(july, 8, 2015)                         1
(thank, sir, i'm)                       1
(identified, hormone, called)           1
(carson, holmquist, advertisement)      1
(helpful, advice, looking)              1
(15, 000, middle)                       1
(majority, legislature, forced)         1
(susan, bay, nimoy)                     1
(diego, part, vegetarian)               1
(saying, since, beginning)              1
(possible, cat, owner)                  1
(stream, behind, niirala)               1
(liberty, prosperity, spoken)           1
(siren's, song, aqua)                   1
(150, doctors, nurses)                  1
(university, study, found)              1
(butter, 1, tablespoon)                 1
(sale, bookings, first)                 1

[173523 rows x 1 columns]

## Build Predictive Model

In [0]:
from sklearn.ensemble import RandomForestRegressor

In [0]:
all_data = dfc.dropna(subset=['img_count', 'word_count'])

In [0]:
all_data.reset_index(inplace=True, drop=True)

In [0]:
all_data

date       fb  \
0   2015-02-23 15:39:00   400000   
1   2015-07-11 00:36:00   402000   
2   2015-06-26 22:08:00   401000   
3   2015-06-23 22:58:00   395000   
4   2015-07-01 18:30:00   392000   
5   2015-08-21 19:40:00   383000   
6   2015-10-15 20:46:00   386000   
7   2015-02-24 23:00:00   383000   
8   2015-11-06 18:10:00   383000   
9   2015-09-25 17:13:00   382000   
10  2015-02-25 23:17:00   373000   
11  2015-03-16 23:20:00   378000   
12  2015-06-17 16:01:00   378000   
13  2015-10-08 21:15:00   369000   
14  2016-02-04 19:53:00   374000   
15  2015-02-20 01:46:00   371000   
16  2015-07-30 04:22:00   359000   
17  2015-07-29 21:38:00   369000   
18  2015-09-20 19:05:00   352000   
19  2015-12-05 03:13:00   366000   
20  2015-06-09 16:31:00   362000   
21  2015-03-15 15:33:00   363000   
22  2015-10-18 21:58:00  1100000   
23  2015-04-28 01:22:00   360000   
24  2015-04-28 01:22:00   358000   
25  2015-12-01 14:31:00   357000   
26  2015-07-06 12:00:00   340000   
27  2015-03-04 13:02:00   349000   
28  2015-03-05 17:26:00   337000   
29  2015-03-24 21:20:00   344000   
..                  ...      ...   
355 2015-07-07 12:02:00   495000   
356 2015-08-26 02:17:00   496000   
357 2016-01-23 23:01:00   496000   
358 2015-06-18 22:56:00   490000   
359 2016-01-10 01:45:00   492000   
360 2015-06-26 21:31:00  1400000   
361 2016-02-02 01:15:00   473000   
362 2015-08-18 17:23:00   466000   
363 2015-07-07 23:10:00   465000   
364 2015-09-30 15:29:00   460000   
365 2015-04-11 20:30:00   436000   
366 2015-10-20 13:03:00   455000   
367 2015-10-25 13:16:00   447000   
368 2015-09-28 23:55:00   448000   
369 2015-10-14 00:42:00   441000   
370 2015-04-06 19:52:00   433000   
371 2015-04-28 02:44:00   427000   
372 2015-04-18 20:41:00   428000   
373 2016-01-12 06:05:00   429000   
374 2015-06-08 02:09:00   402000   
375 2015-06-20 14:20:00   416000   
376 2015-06-25 17:18:00   417000   
377 2015-09-22 23:02:00  1200000   
378 2015-08-05 16:25:00   402000   
379 2015-07-01 18:31:00   401000   
380 2015-03-13 02:27:00   397000   
381 2015-09-24 02:43:00   407000   
382 2015-08-07 01:16:00   409000   
383 2015-11-18 00:36:00   407000   
384 2015-03-05 23:21:00   406000   

                                                                                                                                                                                                        html  \
0    <div>\n<p>\n</p><p><a href="http://www.buzzfeed.com/trending"> <img src="https://img.buzzfeed.com/buzzfeed-static/static/badge_images/viral.png?v=1455385914"></a></p>\n<div>\n<div>\n<div>\n<div>1....   
1    <div>\n<p>\n</p><div>\n<div>\n<div>\n<div>1. Everyone loves ice cream. </div>\n<div>2. And of course churros are just as incredible. </div>\n<div>\n<h2>3. Individually, they make for a pretty tast...   
2    <div>\n<h3>In 2013, Zerlina Maxwell ignited a firestorm of controversy when she strongly recommended we stop telling women how to not get raped. </h3>\n<p>Here are her words, from the <a href="htt...   
3    <div>\n<p>Despite that, Congress still hasn't found the will to deal with gun violence in the U.S. Suffice it to say a lot of people are angry. Including the pile of celebrities you'll see below.<...   
4    <div>\n<div>\n<p><a href="http://www.buzzfeed.com/trending"> <img src="https://img.buzzfeed.com/buzzfeed-static/static/badge_images/viral.png?v=1455385914"></a></p> \n</div>\n<div>\n<p><a href="ht...   
5    <div>\n<p>So two yahoos from Southie in my hometown of Boston severely <a href="http://www.nytimes.com/politics/first-draft/2015/08/20/a-beating-in-boston-said-to-be-inspired-by-donald-trumps-immi...   
6    <div>\n<i>[Editor's note: Anxiety and depression affect everyone differently - but dealing with both is extremely common. <a href="http://www.adaa.org/about-adaa/press-room/facts-statistics">Nearl...   
7    <div>\n<p>The next time your right-wing family member or former high school classmate posts a status update or tweet 

In [0]:
train_index = []
test_index = []
for i in all_data.index:
    result = np.random.choice(2, p=[.65,.35])
    if result == 1:
        test_index.append(i)
    else:
        train_index.append(i)

In [0]:
print('test length:', len(test_index), '\ntrain length:', len(train_index))

test length: 141 
train length: 244


In [0]:
sites = pd.get_dummies(all_data['site'])

In [0]:
sites

ABC News  Asbury Park Press  BBC News  Bloomberg.com  Boredom Therapy  \
0           0                  0         0              0                0   
1           0                  0         0              0                0   
2           0                  0         0              0                0   
3           0                  0         0              0                0   
4           0                  0         0              0                0   
5           0                  0         0              0                0   
6           0                  0         0              0                0   
7           0                  0         0              0                0   
8           0                  0         0              0                0   
9           0                  0         0              0                0   
10          0                  0         1              0                0   
11          0                  0         0              0                0   
12          0                  0         0              0                0   
13          0                  0         0              0                0   
14          0                  0         0              0                0   
15          0                  0         0              0                0   
16          0                  0         0              0                0   
17          0                  0         0              0                0   
18          0                  0         0              0                0   
19          0                  0         0              0                0   
20          0                  0         0              0                0   
21          0                  0         0              0                0   
22          0                  0         0              0                0   
23          0                  0         0              0                0   
24          0                  0         0              0                0   
25          0                  0         0              0                0   
26          0                  0         0              0                0   
27          0                  0         0              0                0   
28          0                  0         0              0                0   
29          0                  0         0              0                0   
..        ...                ...       ...            ...              ...   
355         0                  0         0              0                0   
356         0                  0         0              0                0   
357         0                  0         0              0                0   
358         0                  0         0              0                0   
359         0                  0         0              0                0   
360         0                  0         0              0                0   
361         0                  0         0              0                0   
362         0                  0         0              0                0   
363         0                  0         0              0                0   
364         0                  0         0              0                0   
365         0                  0         0              0                0   
366         0                  0         0              0                0   
367         0                  0         0              0                0   
368         0                  0         0              0                0   
369         0                  0         0              0                0   
370         0                  0         0              0                0   
371         0                  0         0              0                0   
372         0                  0         0              0                0   
373         0                  0         0              0                0   
374         0              

In [0]:
y_train = all_data.iloc[train_index]['fb'].astype(int)
X_train_nosite = all_data.iloc[train_index][['img_count', 'word_count']]

In [0]:
X_train = pd.merge(X_train_nosite, sites.iloc[train_index], left_index=True, right_index=True)

In [0]:
y_test = all_data.iloc[test_index]['fb'].astype(int)
X_test_nosite = all_data.iloc[test_index][['img_count', 'word_count']]

In [0]:
X_test = pd.merge(X_test_nosite, sites.iloc[test_index], left_index=True, right_index=True)

In [0]:
clf = RandomForestRegressor(n_estimators=100)

In [0]:
clf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [0]:
y_pred = clf.predict(X_test)

In [0]:
y_actual = y_test

In [0]:
deltas = pd.DataFrame(list(zip(y_pred, y_actual, (y_pred - y_actual)/(y_actual))), columns=['predicted', 'actual', 'delta'])

In [0]:
deltas

predicted   actual     delta
0    550550.000000   395000  0.393797
1    337070.000000   383000 -0.119922
2    223238.333333   382000 -0.415606
3    251160.000000   369000 -0.319350
4    233130.000000   363000 -0.357769
5    270203.333333   340000 -0.205284
6    221960.000000   349000 -0.364011
7    246920.000000   340000 -0.273765
8    228910.000000   344000 -0.334564
9    326753.000000   339000 -0.036127
10   268330.000000   331000 -0.189335
11   228408.166667   320000 -0.286224
12   563800.000000   330000  0.708485
13   230590.000000   316000 -0.270285
14   269430.000000   975000 -0.723662
15   246348.166667   318000 -0.225320
16   210150.000000   316000 -0.334968
17   415430.000000   312000  0.331506
18   245140.000000   312000 -0.214295
19   317713.333333   958000 -0.668358
20   234720.000000   309000 -0.240388
21   218010.000000   305000 -0.285213
22   216679.333333   299000 -0.275320
23   235040.000000   298000 -0.211275
24   225793.333333   281000 -0.196465
25   235130.000000   293000 -0.197509
26   286020.000000   294000 -0.027143
27   254840.000000   286000 -0.108951
28   299070.000000   286000  0.045699
29   232840.000000   286000 -0.185874
..             ...      ...       ...
111  221630.000000   184000  0.204511
112  226760.000000   180000  0.259778
113  243070.000000   145000  0.676345
114  271420.000000   183000  0.483169
115  401040.000000   184000  1.179565
116  244830.000000   551000 -0.555662
117  278530.000000   182000  0.530385
118  246430.000000   181000  0.361492
119  260875.000000   180000  0.449306
120  318900.000000   171000  0.864912
121  247200.000000   178000  0.388764
122  225740.000000   178000  0.268202
123  227315.000000   169000  0.345059
124  257370.000000   179000  0.437821
125  241720.000000   510000 -0.526039
126  381950.000000   499000 -0.234569
127  224110.000000   496000 -0.548165
128  320610.000000   490000 -0.345694
129  300010.000000  1400000 -0.785707
130  276670.000000   466000 -0.406288
131  209560.000000   460000 -0.544435
132  247860.000000   441000 -0.437959
133  193070.000000   433000 -0.554111
134  409480.000000   427000 -0.041030
135  237480.000000   429000 -0.446434
136  215450.000000   417000 -0.483333
137  378990.000000   402000 -0.057239
138  262860.000000   401000 -0.344489
139  208870.000000   407000 -0.486806
140  464210.000000   406000  0.143374

[141 rows x 3 columns]

In [0]:
deltas['delta'].describe()

count    141.000000
mean       0.059933
std        0.538437
min       -0.785707
25%       -0.265133
50%        0.012279
75%        0.268202
max        2.762419
Name: delta, dtype: float64

In [0]:
a = pd.Series([10,10,10,10])
b = pd.Series([12,8,8,12])

In [0]:
np.sqrt(np.mean((b-a)**2))/np.mean(a)

0.20000000000000001

In [0]:
(b-a).mean()

0.0

In [0]:
np.sqrt(np.mean((y_pred-y_actual)**2))/np.mean(y_actual)

0.66971087381229732

## Use title n-grams

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

In [0]:
vect = CountVectorizer(ngram_range=(1,3))

In [0]:
X_titles_all = vect.fit_transform(all_data['title'])

In [0]:
X_titles_train = X_titles_all[train_index]

In [0]:
X_titles_test = X_titles_all[test_index]

In [0]:
len(X_titles_train.toarray())

244

In [0]:
len(X_titles_test.toarray())

141

In [0]:
len(X_train)

244

In [0]:
len(X_test)

141

In [0]:
X_test = pd.merge(X_test, pd.DataFrame(X_titles_test.toarray(), index=X_test.index), left_index=True, right_index=True)

In [0]:
X_train = pd.merge(X_train, pd.DataFrame(X_titles_train.toarray(), index=X_train.index), left_index=True, right_index=True)

In [0]:
clf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [0]:
y_pred = clf.predict(X_test)

In [0]:
deltas = pd.DataFrame(list(zip(y_pred, y_actual, (y_pred - y_actual)/(y_actual))), columns=['predicted', 'actual', 'delta'])

In [0]:
deltas

predicted   actual     delta
0       280568   395000 -0.289701
1       214380   383000 -0.440261
2       283010   382000 -0.259136
3       220910   369000 -0.401328
4       228000   363000 -0.371901
5       204490   340000 -0.398559
6       216470   349000 -0.379742
7       223090   340000 -0.343853
8       217170   344000 -0.368692
9       228620   339000 -0.325605
10      214340   331000 -0.352447
11      269460   320000 -0.157938
12      260000   330000 -0.212121
13      263670   316000 -0.165601
14      269460   975000 -0.723631
15      207840   318000 -0.346415
16      211820   316000 -0.329684
17      316110   312000  0.013173
18      359080   312000  0.150897
19      263930   958000 -0.724499
20      274400   309000 -0.111974
21      215240   305000 -0.294295
22      209770   299000 -0.298428
23      207590   298000 -0.303389
24      213170   281000 -0.241388
25      206000   293000 -0.296928
26      325140   294000  0.105918
27      305800   286000  0.069231
28      291290   286000  0.018497
29      227310   286000 -0.205210
..         ...      ...       ...
111     249190   184000  0.354293
112     201510   180000  0.119500
113     198070   145000  0.366000
114     280660   183000  0.533661
115     208220   184000  0.131630
116     238440   551000 -0.567260
117     303090   182000  0.665330
118     230910   181000  0.275746
119     215250   180000  0.195833
120     208250   171000  0.217836
121     217690   178000  0.222978
122     205790   178000  0.156124
123     230740   169000  0.365325
124     221060   179000  0.234972
125     209520   510000 -0.589176
126     513030   499000  0.028116
127     249940   496000 -0.496089
128     234290   490000 -0.521857
129     196600  1400000 -0.859571
130     218130   466000 -0.531910
131     215970   460000 -0.530500
132     247810   441000 -0.438073
133     245130   433000 -0.433880
134     204360   427000 -0.521405
135     238110   429000 -0.444965
136     220710   417000 -0.470719
137     309060   402000 -0.231194
138     249180   401000 -0.378603
139     272680   407000 -0.330025
140     251380   406000 -0.380837

[141 rows x 3 columns]

In [0]:
np.sqrt(np.mean((y_pred-y_actual)**2))/np.mean(y_actual)

0.64352892438189691

## Add Title Word Count

In [0]:
all_data = all_data.assign(title_wc = all_data['title'].map(lambda x: len(x.split(' '))))

In [0]:
X_train = pd.merge(X_train, all_data[['title_wc']], left_index=True, right_index=True)

In [0]:
X_test = pd.merge(X_test, all_data[['title_wc']], left_index=True, right_index=True)

In [0]:
clf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [0]:
y_pred = clf.predict(X_test)

In [0]:
y_actual = y_test

In [0]:
np.sqrt(np.mean((y_pred-y_actual)**2))/np.mean(y_actual)

0.64134526362902999